In [1]:
import os, re, csv, math, codecs, logging
from collections import Counter
from pathlib import Path
from io import StringIO
import pickle
import gdown

import numpy as np
from matplotlib import pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# from keras.metrics import F1Score

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))
class_num = 20

In [3]:
# descargamos los embeddings de palabras de Fasttext para inglés y descomprimimos el archivo.
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
!unzip wiki-news-300d-1M.vec.zip

--2024-06-27 01:10:08--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.15, 108.157.254.121, 108.157.254.124, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M   186MB/s    in 3.5s    

2024-06-27 01:10:12 (187 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]

Archive:  wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [4]:
# cargamos los embeddings de palabras
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('wiki-news-300d-1M.vec', encoding='utf-8')

for line in f:
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print(f'found {len(embeddings_index)} word vectors')

loading word embeddings...
found 999995 word vectors


In [15]:
# instanciamos el tokenizador
token = Tokenizer(num_words=300000,
                filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                lower=True,
                split=' ',
                char_level=False,
                oov_token="UNK",
                document_count=0)

In [16]:
# fiteamos el tokenizador
token.fit_on_texts(newsgroups_train.data)

In [17]:
# obtenemos los diccionarios idx2word y word2idx
reverse_dictionary = token.index_word
dictionary = dict([(value, key) for (key, value) in reverse_dictionary.items()])
# CHECK QUE EMPIEZA POR 0

In [19]:
# cargamos en una matriz los embeddings de las palabras
# presentes en el vocabulario
embed_dim=300
num_words=len(dictionary)+1
embedding_matrix=np.zeros([num_words,embed_dim])
for word, idx in dictionary.items():
  if idx <= num_words and word in embeddings_index:
    embedding_matrix[idx,:]=embeddings_index[word]

In [20]:
embedding_matrix.shape

(105374, 300)

In [21]:
# se tokenizan los textos
train_sequences=token.texts_to_sequences(newsgroups_train.data)
test_sequences=token.texts_to_sequences(newsgroups_test.data)

In [67]:
# En este punto seleccionamos el tamaño de contexto a procesar en la variable `max_len`
max_len = 500
train_sequences=pad_sequences(train_sequences,maxlen=max_len)
test_sequences=pad_sequences(test_sequences,maxlen=max_len)

In [68]:
from keras.layers import Bidirectional, LSTM, GRU, Dense, Embedding, Dropout, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.regularizers import L2

In [69]:
model = Sequential()

model.add(Embedding(input_dim=num_words, output_dim=embed_dim, weights=[embedding_matrix], input_length=None, trainable=False))

model.add(Bidirectional(GRU(200, return_sequences=True)))

model.add(Bidirectional(GRU(100)))

model.add(Dense(class_num, activation='softmax'))

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(learning_rate = 0.001), metrics=['accuracy'])

model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, None, 300)         31612200  
                                                                 
 bidirectional_37 (Bidirect  (None, None, 400)         602400    
 ional)                                                          
                                                                 
 bidirectional_38 (Bidirect  (None, 200)               301200    
 ional)                                                          
                                                                 
 dense_33 (Dense)            (None, 20)                4020      
                                                                 
Total params: 32519820 (124.05 MB)
Trainable params: 907620 (3.46 MB)
Non-trainable params: 31612200 (120.59 MB)
_________________________________________________________________


In [70]:
early_stopping = EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=8,
    verbose=0,
    mode="max",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0,
)
reducelr = ReduceLROnPlateau(monitor='val_accuracy',
    patience=2,
    factor=0.5
)

history = model.fit(train_sequences, newsgroups_train.target,
                    batch_size=128,
                    epochs=100,
                    validation_split=0.2,
                    callbacks=[early_stopping, reducelr]
                    )


Epoch 1/100
71/71 [==============================] - 23s 235ms/step - loss: 2.6816 - accuracy: 0.1316 - val_loss: 2.3267 - val_accuracy: 0.2240 - lr: 0.0010
Epoch 2/100
71/71 [==============================] - 16s 223ms/step - loss: 2.0903 - accuracy: 0.2816 - val_loss: 1.9478 - val_accuracy: 0.3243 - lr: 0.0010
Epoch 3/100
71/71 [==============================] - 15s 217ms/step - loss: 1.6809 - accuracy: 0.4183 - val_loss: 1.5567 - val_accuracy: 0.4565 - lr: 0.0010
Epoch 4/100
71/71 [==============================] - 15s 216ms/step - loss: 1.4153 - accuracy: 0.5109 - val_loss: 1.4369 - val_accuracy: 0.5064 - lr: 0.0010
Epoch 5/100
71/71 [==============================] - 15s 214ms/step - loss: 1.2532 - accuracy: 0.5715 - val_loss: 1.3286 - val_accuracy: 0.5479 - lr: 0.0010
Epoch 6/100
71/71 [==============================] - 15s 214ms/step - loss: 1.1325 - accuracy: 0.6247 - val_loss: 1.2236 - val_accuracy: 0.5895 - lr: 0.0010
Epoch 7/100
71/71 [==============================] - 15s 2

In [71]:
# Medir F1-score en test
from sklearn.metrics import f1_score
y_pred = model.predict(test_sequences)
y_pred = np.argmax(y_pred, axis=1)
y_test = newsgroups_test.target
f1_score(y_test, y_pred, average='macro')

236/236 [==============================] - 11s 39ms/step


0.6039619342492413